# Database setup

In [1]:
# %% 
import numpy as np
import sys
import os
# import config
# data
import pandas as pd

from scalenav.plotting import cmap
import scalenav.oop as snoo

import ibis as ib
from ibis import _
import ibis.selectors as s

ib.options.interactive = True
ib.options.graphviz_repr = True

import pypalettes as pypal
import pydeck as pdk

In [2]:
# # print(os.environ)
# print(os.getenv("VSCODE_CWD"))

In [3]:
# import sys
# import os

# print(str(sys.path))

# dir_path = os.path.dirname(os.path.realpath(__file__))
# print("current working dir: %s" % dir_path)

# sys.path.insert(0, dir_path)

In [2]:
from parameters import *

In [3]:
#  %% 
db_filename = "../datasets/global_data.db"
db_test = "../datasets/db_test.db"

In [6]:
# in scalenav
# def sn_connect(database = ':memory:',interactive : bool = True,**kwargs):
#     """Create a duckDB connection with spatial and H3 extensions loaded.
#     This function will not create a db from a jupyter notebook. run it in a script or a terminal to perform this action.
#     """
    
#     ib.options.interactive = interactive

#     if database in ["memory", ':memory:', "mem"]:
#         print("Connecting to a temporary in-memory DB instance.")
#         conn = ib.duckdb.connect()
#     elif os.path.exists(database):
#         print("connecting to existing database.")
#         conn = ib.duckdb.connect(database)
#     else :
#         print("Creating database at'",database,"'.")
#         with duckdb.connect(database) as con:
#             pass
#         conn = ib.duckdb.connect(database)

#     conn.raw_sql("""
#         INSTALL spatial; 
#         LOAD spatial;
#         INSTALL h3 FROM community;
#         LOAD h3;
#                     """)
    
#     return conn


In [7]:
# def sn_head(input : ib.Table,n : int = 5):
#     print(input.count())
#     print(input.head(n))

In [4]:
conn = snoo.sn_connect(database=db_filename,name="db_test",memory_limit="100GB")

connecting to existing database.
connecting to existing database.


In [5]:
type(conn)

ibis.backends.duckdb.Backend

In [9]:
# conn = ib.duckdb.connect(db_filename)
conn.sql("SELECT 42;")

┏━━━━━━━┓
┃ 42    ┃
┡━━━━━━━┩
│ int32 │
├───────┤
│    42 │
└───────┘

In [10]:
table1 = conn.create_table('table1', obj = pd.DataFrame(data={"a" : [1,2,3],
                                                            "b" : ["a","b","c"]}),overwrite=True)

In [11]:
# %%
conn.sql("SELECT * FROM table1;")

ib.to_sql(table1)

```sql
SELECT
  *
FROM "global_data"."main"."table1"
```

In [12]:
conn.list_tables()

['copernicus',
 'dose',
 'kummu',
 'local_gdp',
 'mapsapam',
 'nres',
 'places',
 'predictors',
 'table1']

## First iteration of the data catalogue

Contains a selection of tables to set up baseline modelling workflows

### Data

- DOSE
- GHSL S NRES
- Population : ghsl / worldpop constrined or unconstrained
- Overture/foursquare pois
- Kummu GDP (?)
-  

### Sector layers 

- cement 
- GEM Power production
- Mapspam
- 

In [13]:
# a = [1,2,3,4,5]
# a[-1]

In [14]:
# a bounded area will be covered in this tutorial for ease

In [15]:
bbox=[7.383,11.079,9.877,12.959]

In [16]:
ingestion_params = {"overwrite" : False,
                    "bbox" : bbox}

In [17]:
# /Users/cenv1069/Documents/data/datasets/kummu_etal/GDP_per_capita_PPP_1990_2015_v2.nc
# /Users/cenv1069/Documents/data/datasets/kummu_etal/GDP_PPP_30arcsec_v3.nc

# src_file = "/Users/cenv1069/Documents/data/datasets/kummu_etal/GDP_per_capita_PPP_1990_2015_v2.nc"
# with rs.open(src_file) as src:
#     print(src.profile)
#     print(src.count)
#     band = src.read(1)
#     print(band.shape)

## Data ingestion


In [18]:
# in scalenav
# 
# def sn_table(conn,name : str,path : str,*args,**kwargs):
#     q_startup = "create"
#     if "overwrite" in kwargs.keys():
#         print("overwriting existing")
#         q_startup = "create or replace"

#     if "bbox" in kwargs.keys():
#         print("reading bbox")
#         conn.raw_sql(f"{q_startup} table {name} as (select * from '{path}' where lon>{bbox[0]} AND lon<{bbox[2]} AND lat>{bbox[1]} AND lat<{bbox[3]});")
#         return conn.table(name)
#     else : 
#         try :
#             conn.raw_sql(f"{q_startup} table {name} as (select * from '{path}');")
#             return conn.table(name)
#         except:
#             if name in conn.list_tables():
#                 print("Backend table exists, connected.")
#                 return conn.table(name)
#         return 0
        

In [19]:
# DOSE
# conn.raw_sql("create or replace table dose as (select * from '../datasets/local_data/dose-wdi/0_3/dose_wdi_geo.parquet');")
dose = snoo.sn_table(conn,name="dose",path="../datasets/local_data/dose-wdi/0_3/dose_wdi_geo.parquet")

Backend table exists, connected.


In [20]:
dose.head()

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ gid_0  ┃ country ┃ gid_1   ┃ grp_usd_2015 ┃ services_usd_2015 ┃ manufacturing_usd_2015 ┃ agriculture_usd_2015 ┃ centr                   ┃ geometry                                                                         ┃ color             ┃ radius    ┃ x         ┃ y         ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string │ string  │ string  │ float64      │ float64           │ float64                │ float64              │ geospatial:geometry     │ geospatial:geometry                                                              │ array<int64>      │ float64   │ float64   │ float64   │
├────────┼─────────┼─────────┼──────────────┼───────────────────┼────────────────────────┼──────────────────────┼─────────────────────────┼──────────────────────────────────────────────────────────────────────────────────┼───────────────────┼───────────┼───────────┼───────────┤
│ ALB    │ Albania │ ALB.1_1 │ 4.341915e+08 │      2.009631e+08 │           9.449606e+07 │         8.588401e+07 │ <POINT (20.091 40.629)> │ <POLYGON ((20.379 40.435, 20.377 40.436, 20.373 40.437, 20.369 40.438, 20.36...> │ [238, 97, ... +2] │ 19.888996 │ 20.090775 │ 40.628500 │
│ ALB    │ Albania │ ALB.2_1 │ 3.719427e+08 │      1.721516e+08 │           8.094841e+07 │         7.357106e+07 │ <POINT (20.25 41.589)>  │ <POLYGON ((20.422 41.282, 20.423 41.282, 20.422 41.283, 20.421 41.284, 20.42...> │ [238, 97, ... +2] │ 19.734250 │ 20.249534 │ 41.589105 │
│ ALB    │ Albania │ ALB.3_1 │ 1.113524e+09 │      5.153885e+08 │           2.423439e+08 │         2.202575e+08 │ <POINT (19.63 41.447)>  │ <POLYGON ((19.638 41.341, 19.636 41.337, 19.637 41.335, 19.64 41.33, 19.642 ...> │ [238, 97, ... +2] │ 20.830796 │ 19.630373 │ 41.447333 │
│ ALB    │ Albania │ ALB.4_1 │ 7.954835e+08 │      3.681851e+08 │           1.731265e+08 │         1.573483e+08 │ <POINT (20.186 41.04)>  │ <POLYGON ((20.437 40.849, 20.437 40.848, 20.44 40.847, 20.441 40.841, 20.446...> │ [238, 97, ... +2] │ 20.494461 │ 20.186454 │ 41.040028 │
│ ALB    │ Albania │ ALB.5_1 │ 1.345159e+09 │      6.225993e+08 │           2.927561e+08 │         2.660753e+08 │ <POINT (19.621 40.775)> │ <POLYGON ((19.744 40.452, 19.745 40.457, 19.743 40.459, 19.741 40.462, 19.73...> │ [238, 97, ... +2] │ 21.019778 │ 19.620528 │ 40.774633 │
└────────┴─────────┴─────────┴──────────────┴───────────────────┴────────────────────────┴──────────────────────┴─────────────────────────┴──────────────────────────────────────────────────────────────────────────────────┴───────────────────┴───────────┴───────────┴───────────┘

In [21]:
# DOSE
# conn.raw_sql(" CREATE OR REPLACE TABLE local_gdp as (Select * from ST_READ('../datasets/Local_GDP_25deg/shapefile/geom_0_25deg.shp'));")
local_gdp = snoo.sn_table(conn,name="local_gdp",path='../datasets/Local_GDP_25deg/shapefile/geom_0_25deg.shp')

Backend table exists, connected.


In [22]:
local_gdp.head()

┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ iso    ┃ fid_2    ┃ cell_id ┃ sbcll_d ┃ s__0_25 ┃ geom                                                                             ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string │ float64  │ string  │ int32   │ int32   │ geospatial:geometry                                                              │
├────────┼──────────┼─────────┼─────────┼─────────┼──────────────────────────────────────────────────────────────────────────────────┤
│ AFG    │ 297768.0 │ 18611   │       2 │       4 │ <MULTIPOLYGON (((70.335 38.01, 70.335 38.011, 70.336 38.013, 70.336 38.014, ...> │
│ AFG    │ 297773.0 │ 18611   │       4 │       1 │ <POLYGON ((70.557 38.255, 70.557 38.255, 70.557 38.257, 70.557 38.258, 70.55...> │
│ AFG    │ 297774.0 │ 18611   │       4 │       2 │ <POLYGON ((70.505 38.122, 70.505 38.122, 70.506 38.123, 70.506 38.124, 70.50...> │
│ AFG    │ 297775.0 │ 18611   │       4 │       3 │ <POLYGON ((70.81 38.444, 70.812 38.444, 70.813 38.444, 70.815 38.445, 70.817...> │
│ AFG    │ 297776.0 │ 18611   │       4 │       4 │ <POLYGON ((71 38, 70.75 38, 70.75 38.25, 71 38.25, 71 38))>                      │
└────────┴──────────┴─────────┴─────────┴─────────┴──────────────────────────────────────────────────────────────────────────────────┘

In [23]:
# local_gdp_df = local_gdp.execute(limit=1000).set_crs("epsg:4326")
# local_gdp_df.explore()

In [24]:
# mapspam
mapspam = snoo.sn_table(conn,name="mapsapam",path="../datasets/mapspam/processed/spam_2020_yield_v2.parquet",**ingestion_params)

overwriting existing
reading bbox


In [25]:
# kummu
kummu = snoo.sn_table(conn,name="kummu",path="../datasets/kummu_etal/processed/GDP_PPP_30arcsec/*band_1*.parquet",**ingestion_params)

overwriting existing
reading bbox


In [26]:
snoo.sn_head(kummu)

┌───────┐
│ 20585 │
└───────┘


┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ lon      ┃ lat       ┃ band_var      ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ float32  │ float32   │ float32       │
├──────────┼───────────┼───────────────┤
│ 7.387500 │ 12.954166 │  99221.453125 │
│ 7.429167 │ 12.954166 │  54983.750000 │
│ 7.470833 │ 12.954166 │  50964.957031 │
│ 7.562500 │ 12.954166 │ 564937.187500 │
│ 7.570833 │ 12.954166 │ 997545.125000 │
└──────────┴───────────┴───────────────┘


In [27]:
# GHSL NRES
nres = snoo.sn_table(conn,name="nres",path="../datasets/JRC/processed/S_NRES_10/**.parquet",**ingestion_params)

overwriting existing
reading bbox


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [28]:
# copernicus
copernic_built = snoo.sn_table(conn,name="copernicus",path="/Users/cenv1069/Documents/data/datasets/copernicus/processed/copernicus_builtup_100m_2018/**.parquet",**ingestion_params)

overwriting existing
reading bbox


In [29]:
places = snoo.sn_table(conn,name="places",path="../" + overture_places_landuses_filename,overwrite=True)
places = (
    places
    .filter(
        _.x>bbox[0],
        _.y>bbox[1],
        _.x<bbox[2],
        _.y<bbox[3],)
    .fill_null({"dose" : "uncategorised"})
    .rename({"lon" : "x",
             "lat" : "y"})
)

overwriting existing


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [30]:
places = snoo.sn_project(places,res=8)

Existing h3_id column will be overwritten
Assuming coordinates columns ('lon','lat')


In [31]:
places.select("dose").describe()

┏━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ name   ┃ pos   ┃ type   ┃ count ┃ nulls ┃ unique ┃ mode     ┃
┡━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ string │ int16 │ string │ int64 │ int64 │ int64  │ string   │
├────────┼───────┼────────┼───────┼───────┼────────┼──────────┤
│ dose   │     0 │ string │  5069 │     0 │      3 │ services │
└────────┴───────┴────────┴───────┴───────┴────────┴──────────┘

In [32]:
snoo.sn_head(places)

┌──────┐
│ 5069 │
└──────┘


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━
┃ id                               ┃ sec_cat  ┃ lon      ┃ lat       ┃ sec_cat_right ┃ section ┃ isic_embed ┃ isic_
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━
│ string                           │ string   │ float32  │ float32   │ string        │ string  │ string     │ strin
├──────────────────────────────────┼──────────┼──────────┼───────────┼───────────────┼─────────┼────────────┼──────
│ 08b58195651b4fff0002bf0759a69f80 │ pitch    │ 8.033602 │ 11.360646 │ pitch         │ R       │ 92         │ Gambl
│ 08b5819575246fff0002b747e62a099c │ farmland │ 8.008574 │ 11.319696 │ farmland      │ A       │ 01         │ Crop 
│ 08b581957535efff0002b803cce36049 │ farmland │ 8.006901 │ 11.321367 │ farmland      │ A       │ 01         │ Crop 
│ 08b5819575358fff0002beff76bcff23 │ farmland │ 8.007175 │ 11.321362 │ f

In [33]:
places.head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ id                               ┃ sec_cat  ┃ lon      ┃ lat       ┃ sec_cat_right ┃ section ┃ isic_embed ┃ isic_descr                                                         ┃ dose        ┃ match_score ┃ h3_id           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string                           │ string   │ float32  │ float32   │ string        │ string  │ string     │ string                                                             │ string      │ float64     │ string          │
├──────────────────────────────────┼──────────┼──────────┼───────────┼───────────────┼─────────┼────────────┼────────────────────────────────────────────────────────────────────┼─────────────┼─────────────┼─────────────────┤
│ 08b58195651b4fff0002bf0759a69f80 │ pitch    │ 8.033602 │ 11.360646 │ pitch         │ R       │ 92         │ Gambling and betting activities                                    │ services    │    0.221822 │ 885819565dfffff │
│ 08b5819575246fff0002b747e62a099c │ farmland │ 8.008574 │ 11.319696 │ farmland      │ A       │ 01         │ Crop and animal production, hunting and related service activities │ agriculture │    0.506969 │ 8858195753fffff │
│ 08b581957535efff0002b803cce36049 │ farmland │ 8.006901 │ 11.321367 │ farmland      │ A       │ 01         │ Crop and animal production, hunting and related service activities │ agriculture │    0.506969 │ 8858195753fffff │
│ 08b5819575358fff0002beff76bcff23 │ farmland │ 8.007175 │ 11.321362 │ farmland      │ A       │ 01         │ Crop and animal production, hunting and related service activities │ agriculture │    0.506969 │ 8858195753fffff │
│ 08b5819575262fff0002b06e77f429bd │ farmland │ 8.008208 │ 11.320755 │ farmland      │ A       │ 01         │ Crop and animal production, hunting and related service activities │ agriculture │    0.506969 │ 8858195753fffff │
└──────────────────────────────────┴──────────┴──────────┴───────────┴───────────────┴─────────┴────────────┴────────────────────────────────────────────────────────────────────┴─────────────┴─────────────┴─────────────────┘

In [34]:
conn.list_tables()

['copernicus',
 'dose',
 'kummu',
 'local_gdp',
 'mapsapam',
 'nres',
 'places',
 'predictors',
 'table1']

In [ ]:
places_h3_dose = snoo.sn_reindex(places.select("dose","h3_id","id"),names_from="dose",values_from="id")

In [138]:
tab_name = "places_h3_dose"
places_h3_dose = conn.create_table(obj=places_h3_dose,name=tab_name,overwrite=True)

In [ ]:
conn.raw_sql(f"CREATE INDEX h3_idx on {tab_name} (h3_id);")

In [140]:
conn.list_tables()

['copernicus',
 'dose',
 'kummu',
 'local_gdp',
 'mapsapam',
 'nres',
 'places',
 'places_h3_dose',
 'predictors',
 'table1']

In [141]:
places_h3_dose.head()

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ h3_id           ┃ manufacturing ┃ agriculture ┃ services ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ string          │ int64         │ int64       │ int64    │
├─────────────────┼───────────────┼─────────────┼──────────┤
│ 8858182c31fffff │             0 │           0 │        2 │
│ 8858182a91fffff │             1 │           0 │        0 │
│ 88581821e5fffff │             0 │           1 │        0 │
│ 8858192e83fffff │             0 │           0 │        3 │
│ 8858e42b5bfffff │             0 │           0 │        1 │
└─────────────────┴───────────────┴─────────────┴──────────┘

In [142]:
places_h3_section = snoo.sn_reindex(places.select("section","h3_id","id"),names_from="section",values_from="id")

In [143]:
snoo.sn_head(places_h3_section)

┌─────┐
│ 965 │
└─────┘


┏━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳
┃ h3_id           ┃ E     ┃ I     ┃ N     ┃ J     ┃ G     ┃ F     ┃ R     ┃ P     ┃ B     ┃ M     ┃ A     ┃ S     ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇
│ string          │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │
├─────────────────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼
│ 885819364bfffff │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 8858e4526bfffff │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 885819b2a7fffff │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     1 │
│ 88580a4c0bfffff │     0 │     0 │     0 │     0 │     0 │     0 │     

## Tables

In [144]:
copernic_built.head()

┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ lon      ┃ lat       ┃ band_var ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ float32  │ float32   │ float32  │
├──────────┼───────────┼──────────┤
│ 7.385417 │ 12.958829 │      9.0 │
│ 7.386409 │ 12.958829 │      6.0 │
│ 7.387401 │ 12.958829 │      9.0 │
│ 7.388393 │ 12.958829 │     88.0 │
│ 7.389385 │ 12.958829 │     52.0 │
└──────────┴───────────┴──────────┘

In [145]:
nres.head()

┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ lon      ┃ lat       ┃ band_var ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ float32  │ float32   │ float32  │
├──────────┼───────────┼──────────┤
│ 9.832885 │ 12.908677 │      1.0 │
│ 9.832885 │ 12.908578 │      1.0 │
│ 9.832985 │ 12.908578 │      1.0 │
│ 9.833084 │ 12.908578 │      1.0 │
│ 9.832786 │ 12.908478 │      1.0 │
└──────────┴───────────┴──────────┘

In [146]:
kummu.head()

┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ lon      ┃ lat       ┃ band_var      ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ float32  │ float32   │ float32       │
├──────────┼───────────┼───────────────┤
│ 7.387500 │ 12.954166 │  99221.453125 │
│ 7.429167 │ 12.954166 │  54983.750000 │
│ 7.470833 │ 12.954166 │  50964.957031 │
│ 7.562500 │ 12.954166 │ 564937.187500 │
│ 7.570833 │ 12.954166 │ 997545.125000 │
└──────────┴───────────┴───────────────┘

In [147]:
mapspam.head()

┏━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ lon      ┃ lat       ┃ band_var    ┃
┡━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ float32  │ float32   │ float32     │
├──────────┼───────────┼─────────────┤
│ 8.541666 │ 12.958334 │  401.399994 │
│ 8.708333 │ 12.958334 │ 1363.099976 │
│ 8.791666 │ 12.958334 │ 1448.500000 │
│ 8.874999 │ 12.958334 │ 1429.599976 │
│ 8.958333 │ 12.958334 │ 1403.099976 │
└──────────┴───────────┴─────────────┘

In [148]:
places_h3_section.head()

┏━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃ h3_id           ┃ E     ┃ I     ┃ N     ┃ J     ┃ G     ┃ F     ┃ R     ┃ P     ┃ B     ┃ M     ┃ A     ┃ S     ┃ L     ┃ Q     ┃ C     ┃ K     ┃ H     ┃ D     ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ string          │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │
├─────────────────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ 88581b966dfffff │     0 │     1 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     1 │     0 │     0 │     0 │     0 │
│ 8858182c61fffff │     0 │     1 │     0 │     0 │     2 │     6 │     2 │     3 │     0 │     0 │     0 │     1 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88581838b3fffff │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     1 │     0 │     0 │     0 │
│ 885818213dfffff │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     1 │     0 │     0 │     0 │
│ 88581b9645fffff │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     1 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
└─────────────────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┘

In [149]:
renames =  { x + "_var" : x for x in places_h3_section.columns if x != "h3_id"}

In [150]:
places_h3 = places_h3_section.rename(renames)

In [151]:
places_h3.head()

┏━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃ h3_id           ┃ E_var ┃ I_var ┃ N_var ┃ J_var ┃ G_var ┃ F_var ┃ R_var ┃ P_var ┃ B_var ┃ M_var ┃ A_var ┃ S_var ┃ L_var ┃ Q_var ┃ C_var ┃ K_var ┃ H_var ┃ D_var ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ string          │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │
├─────────────────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ 88580e4b69fffff │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     1 │     1 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580e5967fffff │     0 │     0 │     0 │     0 │     0 │     0 │     5 │     1 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580e4963fffff │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     4 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580e496bfffff │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     4 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 885819af29fffff │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     1 │     0 │     0 │     0 │     0 │
└─────────────────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┘

## Putting the layers together

In [152]:
predictor_layers = {
    # "copernic" : copernic_built,
    "nres" : nres,
    "mapspam" : mapspam,
                    }

In [153]:
predictor_layers_h3 = { n : snoo.sn_project(x,res=8,keep=False) for n,x in predictor_layers.items() }
predictor_layers_h3["places"] = places_h3

Assuming coordinates columns ('lon','lat')
Assuming coordinates columns ('lon','lat')


In [154]:
list(predictor_layers_h3.keys())

['nres', 'mapspam', 'places']

In [155]:
top_layers = [dose,]

In [156]:
constr_layers = [kummu,]

In [157]:
# in scalenav
def sn_combine(conn, input : [ib.Table], name : str, overwrite : bool = False):
    """Provide a dict with named tables, the keys will be reused to create variable names in the binded data. 
    Similar effect to rbind in R.
    """
    id_col = ib.union(*[tab.select("h3_id") for tab in list(input.values())],distinct=True)
    
    for (nam,tab) in input.items():
        if "band_var" in tab.columns:
            tab = tab.rename({nam : "band_var"})
            id_col = id_col.join(tab,how="left",predicates="h3_id",rname="{name}_var").rename({nam+"_var" : nam}).drop(s.matches("h3_id_var"))
        else :
            id_col = id_col.join(tab,how="left",predicates="h3_id",rname="{name}_var").drop(s.matches("h3_id_var"))
    
    id_col_vars = id_col.select(s.matches("_var$")).columns

    return conn.create_table(obj=id_col.fill_null({col : 0 for col in id_col_vars}),name=name,overwrite=overwrite)#

In [158]:
pred_layers = sn_combine(conn,input=predictor_layers_h3,name="predictors",overwrite=True).distinct()

In [54]:
print(pred_layers.count())
pred_layers

┌───────┐
│ 21641 │
└───────┘


┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃ h3_id           ┃ nres_var ┃ mapspam_var ┃ E_var ┃ I_var ┃ M_var ┃ N_var ┃ J_var ┃ F_var ┃ G_var ┃ A_var ┃ S_var ┃ L_var ┃ Q_var ┃ C_var ┃ K_var ┃ H_var ┃ D_var ┃ P_var ┃ R_var ┃ B_var ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ string          │ float32  │ float32     │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │ int64 │
├─────────────────┼──────────┼─────────────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┼───────┤
│ 88580b9717fffff │      1.0 │         0.0 │     0 │     1 │     1 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     2 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580ac35dfffff │      1.0 │         0.0 │     0 │     0 │     0 │     0 │     0 │     0 │     1 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     1 │     0 │     0 │
│ 88580b96a1fffff │      1.0 │         0.0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580b973bfffff │      1.0 │         0.0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580bd2bdfffff │      1.0 │         0.0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580a8941fffff │      1.0 │         0.0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580ac61bfffff │      1.0 │         0.0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580ada9bfffff │      1.0 │         0.0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580a1327fffff │      1.0 │         0.0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ 88580ad9c1fffff │      1.0 │         0.0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │     0 │
│ …               │        … │           … │     … │     … │     … │     … │     … │     … │     … │     … │     … │     … │     … │     … │     … │     … │     … │     … │     … │     … │
└─────────────────┴──────────┴─────────────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┘

In [55]:
conn.list_tables()

['copernicus',
 'dose',
 'kummu',
 'local_gdp',
 'mapsapam',
 'nres',
 'places',
 'predictors',
 'table1']

## Plotting

To keep track of what plot

In [124]:
import json

isic_codes = json.load(open("data/isic_dict.json","r"))

In [129]:
plotting_var = "E"

In [133]:
print("A :", isic_codes["Section A"]["description"])
print("E :", isic_codes["Section E"]["description"])
print("Q :", isic_codes["Section Q"]["description"])
print("B :", isic_codes["Section B"]["description"])

A : Agriculture, forestry and fishing
This section includes the exploitation of vegetal and animal natural resources, comprising the
activities of growing of crops, raising and breeding of animals, harvesting of timber and other
plants, animals or animal products from a farm or their natural habitats.
E : Water supply; sewerage, waste management and
remediation activities
This section includes activities related to the management (including collection, treatment and
disposal) of various forms of waste, such as solid or non-solid industrial or household waste, as
well as contaminated sites. The output of the waste or sewage treatment process can either be
disposed of or become an input into other production processes. Activities of water supply are
also grouped in this section, since they are often carried out in connection with, or by units also
engaged in, the treatment of sewage.
Q : Human health and social work activities
This section includes the provision of health and social work

In [117]:
places.filter(_.section==plotting_var).sec_cat_right.value_counts()

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ sec_cat_right ┃ sec_cat_right_count ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ string        │ int64               │
├───────────────┼─────────────────────┤
│ hospital      │                 159 │
│ eat_and_drink │                  11 │
└───────────────┴─────────────────────┘

In [118]:
pred_layers.agg(var_sum = _[plotting_var+"_var"].sum())

┏━━━━━━━━━┓
┃ var_sum ┃
┡━━━━━━━━━┩
│ int64   │
├─────────┤
│     257 │
└─────────┘

In [119]:
pred_layers_plot_df = snoo.sn_change_res(pred_layers.select("h3_id",plotting_var + "_var"),levels=-1).filter(_[plotting_var+"_var"] > 0).execute()

In [120]:
pred_layers_plot_df.shape

(136, 2)

In [121]:
count_pal = pypal.load_cmap("viridis")
pred_layers_plot_df["cols"] = cmap(pred_layers_plot_df[plotting_var+"_var"],palette=count_pal,log=True)

Max input : 4.04, palette colors : 10


In [123]:

# Define a layer to display on a map
layer = pdk.Layer(
    "H3HexagonLayer",
    pred_layers_plot_df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=False,
    get_hexagon="h3_id",
    get_fill_color="cols",
    get_line_color=[255, 255, 255, 0],
    line_width_min_pixels=1,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=np.mean(bbox[1::2]), longitude=np.mean(bbox[0::2]), zoom=9, bearing=0, pitch=30)


# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "Layer value: {Q_var}"})
r.to_html("../deck_maps/h3_predictor_layer.html", open_browser=True)